# Import Library

In [2]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

---

In [3]:
root = os.path.join(os.getcwd(), 'DATA')

In [4]:
from datetime import date, timedelta


def set_week(df, date):
    '''
    df : datetime 형식의 컬럼을 가지고 있는 dataframe
    date : df에서 datetime 형식을 가진 컬럼명
    return : date의 연도 컬럼과 주차 컬럼을 추가한 dataframe
    '''
    df[date] = pd.to_datetime(df[date])
    df[date] = df[date].dt.date
    df['year'] = df.apply(func=lambda x: x[date].isocalendar()[0], axis=1)
    df['week'] = df.apply(func=lambda x: x[date].isocalendar()[1], axis=1)
    df.drop(date, axis=1, inplace=True)
    

def check_week(df):
    '''
    df에 date가 전부 있는지 확인
    '''
    cnt = 0
    sdate = date(2015, 12, 28)   # start date
    edate = date(2019, 12, 30)   # end date
    delta = edate - sdate       # as timedelta
    mem = set()
    
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        year, week = day.isocalendar()[0], day.isocalendar()[1]
        if year * 100 + week in mem:
          continue
        mem.add(year * 100 + week)
        if df[(df['year'] == year) & (df['week'] == week)].empty:
          print((year, week), end="")
          cnt += 1
    if cnt > 0:
        print()
    print("missing", cnt, "values")    

# Training Data
빅콘테스트 제공 데이터 전처리

## Import Data

In [8]:
df_raw = pd.read_excel(os.path.join(root, 'train.xlsx'))
df_train = df_raw.copy()

### One hot encoding
P_IMPORT_TYPE 이라는 특수한 컬럼에 대한 전처리

In [9]:
import_type_list = set()
for tmp in df_train.P_IMPORT_TYPE.unique():
    for a in tmp.split(','):
        import_type_list.add(a)

In [10]:
for name in import_type_list:
    df_train[name] = 0
    df_train.loc[df_train['P_IMPORT_TYPE'].str.contains(name, regex=False), name] = 1

### Add Columns

In [11]:
set_week(df_train, 'REG_DATE')

In [12]:
check_week(df_train)

(2017, 2)
missing 1 values


In [13]:
# 어종별 수 컬럼 추가
df_train['name_cnt'] = 0;
value_dict = {}
for name, value in zip(df_train['P_NAME'].value_counts().index,df_train['P_NAME'].value_counts()):
    value_dict[name] = value

def value(col):
    return value_dict[col]

df_train['name_cnt'] = df_train['P_NAME'].apply(value)

In [14]:
# 가공 여부 컬럼 추가
df_train['is_processed'] = (df_train['CTRY_1'] != df_train['CTRY_2'])

In [15]:
df_train['import_cnt'] = 0
for i, row in df_train.iterrows():
    for name in import_type_list:
        if row[name] == 1:
            df_train.at[i, 'import_cnt'] += 1

---

# Weather data

## Import Data

In [16]:
df_weather_code = pd.read_csv(os.path.join(root, 'raw_weather_code.csv'), header=0, index_col=0)

In [17]:
weather_list = [pd.read_csv(os.path.join(root, 'raw_weather_20151228_20161227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20161228_20171227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20171228_20181227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20181228_20191227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20191228_20201227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20201228_20210818.csv'), encoding='euc-kr')]


## Preprocess
- '지점'에 따른 나라명 컬럼(CTRY_1)과 해안가여부(is_waterfront) 추가
- 각 나라, 일자 별로 평균 강수량, 풍속, 기온 계산

In [18]:
# 지점에 따라 나라명 추가
def set_country(row):
    data = df_weather_code[df_weather_code['지점'] == row['지점']]
    if data.empty:
        return ""
    return data.iloc[0]['국가명']


def set_waterfront(row):
    data = df_weather_code[df_weather_code['지점'] == row['지점']]
    if data.empty or data.iloc[0]['해안가여부'] != 1:
        return False
    return True
    

def preprocess_weather(df):
    df = df[(df['강수량'] >= 0) & (df['풍속'] >= 0)] # 이상치/결측치 제거
    set_week(df, '일시')  # 날짜 정보 처리
    
    # 1차 평균
    columns = ['year', 'week', '지점']
    df['rain'] = df[columns + ['강수량']].groupby(columns).transform('mean')
    df['wind'] = df[columns + ['풍속']].groupby(columns).transform('mean')
    df['temperature'] = df[columns + ['기온']].groupby(columns).transform('mean')
    
    # 나라명 추가
    df['CTRY_1'] = ""
    for i, row in df.iterrows():
        df.at[i, 'CTRY_1'] = set_country(row)
    
    # 해안가 여부 추가
    df['is_waterfront'] = False
    for i, row in df.iterrows():
        df.at[i, 'is_waterfront'] = set_waterfront(row)
        
#     df = df[df['is_waterfront']]  # 해안가가 아닌 데이터 제외
    
    # 2차 평균
    columns = ['year', 'week', 'CTRY_1']
    df['rain'] = df[columns + ['강수량']].groupby(columns).transform('mean')
    df['wind'] = df[columns + ['풍속']].groupby(columns).transform('mean')
    df['temperature'] = df[columns + ['기온']].groupby(columns).transform('mean')

    # 컬럼/행 정리
    df.drop(columns=['지점명', '지점', '강수량', '풍속', '기온', 'is_waterfront'], inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [19]:
for i in range(len(weather_list)):
    weather_list[i] = preprocess_weather(weather_list[i])

In [39]:
df_weather = pd.concat(weather_list)

## Check

In [40]:
df_weather.describe()

,year,week,rain,wind,temperature
count,1679.000000,1679.000000,1679.000000,1679.000000,1679.000000
mean,2018.232877,25.856462,5.981158,3.642640,14.906772
std,1.607481,15.124088,32.371218,3.200572,10.536941
min,2015.000000,1.000000,0.000000,0.000000,-21.600000
25%,2017.000000,13.000000,1.489908,1.869608,8.266627
50%,2018.000000,25.000000,2.750000,2.452555,15.382857
75%,2020.000000,39.000000,6.504304,5.111601,25.430698
max,2021.000000,53.000000,915.000000,60.000000,30.100000


In [41]:
for country in df_weather['CTRY_1'].unique():
    if country == "":
        continue
    print()
    print(country, "총", len(df_weather[df_weather['CTRY_1'] == country]), "개")
    check_week(df_weather[df_weather['CTRY_1'] == country])


노르웨이 총 236 개
missing 0 values

태국 총 297 개
(2016, 11)
missing 1 values

베트남 총 298 개
(2016, 6)
missing 1 values

중국 총 299 개
missing 0 values

페루 총 7 개
(2015, 53)(2016, 1)(2016, 2)(2016, 3)(2016, 4)(2016, 5)(2016, 6)(2016, 7)(2016, 8)(2016, 9)(2016, 10)(2016, 11)(2016, 12)(2016, 13)(2016, 14)(2016, 15)(2016, 16)(2016, 17)(2016, 18)(2016, 19)(2016, 20)(2016, 21)(2016, 22)(2016, 23)(2016, 24)(2016, 25)(2016, 26)(2016, 27)(2016, 28)(2016, 29)(2016, 30)(2016, 31)(2016, 32)(2016, 33)(2016, 34)(2016, 35)(2016, 36)(2016, 37)(2016, 38)(2016, 39)(2016, 40)(2016, 41)(2016, 42)(2016, 43)(2016, 44)(2016, 45)(2016, 46)(2016, 47)(2016, 48)(2016, 49)(2016, 51)(2016, 52)(2017, 1)(2017, 2)(2017, 3)(2017, 4)(2017, 5)(2017, 6)(2017, 7)(2017, 8)(2017, 9)(2017, 10)(2017, 12)(2017, 13)(2017, 14)(2017, 15)(2017, 16)(2017, 17)(2017, 18)(2017, 19)(2017, 20)(2017, 21)(2017, 22)(2017, 23)(2017, 24)(2017, 25)(2017, 26)(2017, 27)(2017, 28)(2017, 29)(2017, 30)(2017, 31)(2017, 32)(2017, 33)(2017, 34)(2017, 35)(2017, 3

## Filling Missing Values

- 노르웨이, 중국은 결측치 없음
- 태국, 베트남, 칠레는 결측치 1개
> 전/차주 데이터 평균으로 채워넣기
- 페루는 결측치 299개
> 사용 불가

In [42]:
def get_avg(year, week, country, value):
  return (df_weather[(df_weather['year'] == year) & (df_weather['week'] == week + 1) & (df_weather['CTRY_1'] == country)].iloc[0][value] 
          + df_weather[(df_weather['year'] == year) & (df_weather['week'] == week - 1) & (df_weather['CTRY_1'] == country)].iloc[0][value]) / 2

In [43]:
df_weather = df_weather.append({'year': 2016,
                                'week': 11,
                                'CTRY_1': '태국',
                                'rain': get_avg(2016, 11, '태국', 'rain'),
                                'wind': get_avg(2016, 11, '태국', 'wind'),
                                'temperature': get_avg(2016, 11, '태국', 'temperature')},
                               ignore_index=True)
df_weather = df_weather.append({'year': 2016,
                                'week': 6,
                                'CTRY_1': '베트남',
                                'rain': get_avg(2016, 6, '베트남', 'rain'),
                                'wind': get_avg(2016, 6, '베트남', 'wind'),
                                'temperature': get_avg(2016, 6, '베트남', 'temperature')},
                               ignore_index=True)
df_weather = df_weather.append({'year': 2016,
                                'week': 5,
                                'CTRY_1': '칠레',
                                'rain': get_avg(2016, 5, '칠레', 'rain'),
                                'wind': get_avg(2016, 5, '칠레', 'wind'),
                                'temperature': get_avg(2016, 5, '칠레', 'temperature')},
                               ignore_index=True)

for country in ["태국", "베트남", "칠레"]:
    print()
    print(country, "총", len(df_weather[df_weather['CTRY_1'] == country]), "개")
    check_week(df_weather[df_weather['CTRY_1'] == country])


태국 총 298 개
missing 0 values

베트남 총 299 개
missing 0 values

칠레 총 298 개
missing 0 values


## Add to Training Data

In [46]:
df_train = pd.merge(df_train, df_weather, how='left', on=['year', 'week', 'CTRY_1'])

---

# Salinity

## Import Data

In [ ]:
# df_salinity = pd.read_csv(os.path.join(root, 'raw_salinity.csv'))

## Drop Columns & Rows

In [ ]:
# df_salinity.drop(df_salinity.columns[2], inplace=True, axis=1)
# df_salinity = df_salinity[(2015 <= df_salinity['obs_year']) & (df_salinity['obs_year'] <= 2021)]

2020, 2021 데이터의 부재로 인해 보류

---

# Oil

## Import Data

In [111]:
df_oil = pd.read_csv(os.path.join(root, 'raw_oil.csv'))
df_oil_dubai = pd.read_csv(os.path.join(root, 'raw_oil_dubai.csv'))
df_oil_brent = pd.read_csv(os.path.join(root, 'raw_oil_brent.csv'))

## Preprocess

In [112]:
from datetime import datetime
def preprocess_oil(df):
    df['date'] = pd.to_datetime(df['날짜'].str[:4] + df['날짜'].str[6:8] + df['날짜'].str[10:12])
    set_week(df, 'date')  # 날짜 데이터 정리
    df['oil'] = df[['year', 'week', '종가']].groupby(['year', 'week']).transform('mean')
    df.drop(columns = ['날짜', '오픈', '고가', '저가', '거래량', '변동 %', '종가'], inplace=True, axis=1)
    df.drop_duplicates(inplace=True)
    return df

In [113]:
df_oil = preprocess_oil(df_oil)
df_oil_dubai = preprocess_oil(df_oil_dubai)
df_oil_brent = preprocess_oil(df_oil_brent)

## Check

In [131]:
for i in range(1, 53):
  if i not in (list(df_oil_brent[df_oil_brent['year'] == 2019].sort_values(by=['year', 'week'])['week'])):
    print(i)

In [114]:
check_week(df_oil)
check_week(df_oil_dubai)
check_week(df_oil_brent)

missing 0 values
missing 0 values
missing 0 values


모든 데이터가 결측치는 없지만,  
df_oil 에 해당하는 wti 종가가 가장 예민하게 반응하는 값이므로  
이를 사용하도록 한다

## Add to Training Data

In [132]:
df_train = pd.merge(df_train, df_oil, how='left', on=['year', 'week'])

---

# Korea Weather

## Import Data

In [134]:
df_weather_kr = pd.read_csv(os.path.join(root, 'raw_weather_korea.csv'), encoding='euc-kr')

## Preprocess Date

In [136]:
df_weather_kr['date'] = pd.to_datetime(df_weather_kr['일시'].str[:4] + df_weather_kr['일시'].str[5:7] + df_weather_kr['일시'].str[8:])
set_week(df_weather_kr, 'date')

In [137]:
df_weather_kr.sample(5)

,지점,일시,평균 풍속(m/s),평균 기온(°C),평균 수온(°C),year,week
26211,22187,2018-02-14,9.3,12.6,15.8,2018,7
11323,22105,2019-12-21,6.4,7.9,17.2,2019,51
14597,22107,2018-02-03,NaN,3.4,14.9,2018,5
25327,22186,2021-02-01,6.2,7.4,7.5,2021,5
18648,22183,2018-06-19,1.8,20.3,20.0,2018,25


## Group
일자 별로 평균 구하기

In [138]:
df_weather_kr['wind_kr'] = df_weather_kr[['year', 'week', '평균 풍속(m/s)']].groupby(['year', 'week']).transform('mean')
df_weather_kr['temperature_kr'] = df_weather_kr[['year', 'week', '평균 기온(°C)']].groupby(['year', 'week']).transform('mean')
df_weather_kr['water_temp_kr'] = df_weather_kr[['year', 'week', '평균 수온(°C)']].groupby(['year', 'week']).transform('mean')

## Drop Column
어차피 전부 해안가 대한민국이라 지점은 필요없다  
사용한 컬럼은 제거한다

In [139]:
drop = ['지점', '일시', '평균 풍속(m/s)', '평균 기온(°C)', '평균 수온(°C)']

In [140]:
df_weather_kr.drop(columns=drop, inplace=True, axis=1)
df_weather_kr.drop_duplicates(inplace=True)

## Check

In [142]:
check_week(df_weather_kr)

missing 0 values


## Add to Training Data

In [143]:
df_train = pd.merge(df_train, df_weather_kr, how='left', on=['year', 'week'])

---

# 소비자물가지수

## Import Data

In [ ]:
cpi = pd.read_csv(os.path.join(root, 'raw_cpi_purpose.csv'), encoding='cp949')  # CPI

## 소비자물가 총지수

### Drop Columns/Rows

In [ ]:
cpi_purpose.drop('시도별',axis=1,inplace=True)

cpi_purpose_copy = cpi_purpose.copy()

cpi_purpose_copy.drop('지출목적별',axis=1,inplace=True)

cpi_total = cpi_purpose_copy.iloc[0,:] # 전국 소비자물가 총지수만 추출

cpi_total = pd.DataFrame(cpi_total.values,columns=['cpi'],index=cpi_total.index)

cpi_total

### Pivoting

In [ ]:
cpi_purpose_copy = cpi_purpose.copy()

cpi_purpose_copy.drop('지출목적별',axis=1,inplace=True)

cpi_total = cpi_purpose_copy.iloc[0,:] # 전국 소비자물가 총지수만 추출

cpi_total = pd.DataFrame(cpi_total.values,columns=['cpi'],index=cpi_total.index)

cpi_total

## 지출목적별 소비자물가지수

In [ ]:
cpi_purpose2 = cpi_purpose.iloc[[1,11],:]

cpi_purpose2 # 전국 식료품별, 음식서비스별 cpi만 추출

,지출목적별,2015. 12,2016. 01,2016. 02,2016. 03,2016. 04,2016. 05,2016. 06,2016. 07,2016. 08,2016. 09,2016. 10,2016. 11,2016. 12,2017. 01,2017. 02,2017. 03,2017. 04,2017. 05,2017. 06,2017. 07,2017. 08,2017. 09,2017. 10,2017. 11,2017. 12,2018. 01,2018. 02,2018. 03,2018. 04,2018. 05,2018. 06,2018. 07,2018. 08,2018. 09,2018. 10,2018. 11,2018. 12,2019. 01,2019. 02,2019. 03,2019. 04,2019. 05,2019. 06,2019. 07,2019. 08,2019. 09,2019. 10,2019. 11,2019. 12,2020. 01,2020. 02,2020. 03,2020. 04,2020. 05,2020. 06,2020. 07,2020. 08,2020. 09,2020. 10,2020. 11,2020. 12,2021. 01,2021. 02,2021. 03,2021. 04,2021. 05,2021. 06
1,01 식료품 · 비주류음료,99.95,100.76,103.43,102.25,102.30,101.24,99.66,99.65,101.02,105.94,104.48,102.93,104.02,107.16,107.26,106.32,105.19,104.85,104.09,104.39,107.5,108.84,106.16,103.20,104.44,106.00,109.07,107.32,107.86,106.85,105.48,105.80,111.34,115.20,112.16,108.67,108.86,108.8,109.54,108.48,109.4,108.83,107.64,106.68,107.70,110.52,110.74,107.89,109.01,110.81,110.27,111.26,111.29,111.48,111.18,111.28,114.78,119.68,119.82,115.35,115.79,118.04,120.97,120.61,120.34,119.70,118.43
11,11 음식 및 숙박,101.06,101.40,101.67,101.98,102.26,102.40,102.50,102.74,102.91,102.92,103.02,103.07,103.27,103.75,104.02,104.31,104.49,104.80,104.87,105.20,105.5,105.36,105.57,105.67,106.07,106.55,106.94,107.29,107.70,108.07,108.17,108.52,108.76,108.58,108.74,108.96,109.35,109.7,109.89,109.63,109.8,110.01,110.11,110.35,110.64,110.09,110.27,110.29,110.45,110.97,110.93,110.82,110.90,110.86,110.95,111.13,111.35,111.25,111.51,111.41,111.67,111.96,112.26,112.42,112.93,113.11,113.37


In [ ]:
cpi_purpose3 = pd.DataFrame(cpi_purpose2.iloc[0,:].values,index = cpi_purpose2.columns,columns=['식료품 cpi'])

cpi_purpose3['음식 및 숙박 cpi'] = cpi_purpose2.iloc[1,:].values

cpi_purpose3.drop('지출목적별',axis=0,inplace=True) # 지출목적별 행 삭제

cpi_purpose3 # 식료품, 음식 cpi

,식료품 cpi,음식 및 숙박 cpi
2015. 12,99.95,101.06
2016. 01,100.76,101.4
2016. 02,103.43,101.67
2016. 03,102.25,101.98
2016. 04,102.3,102.26
...,...,...
2021. 02,120.97,112.26
2021. 03,120.61,112.42
2021. 04,120.34,112.93
2021. 05,119.7,113.11


## Preprocess Date

In [ ]:
cpi_total.reset_index(inplace=True)  # 날짜 인덱스 > 컬럼으로 변경
cpi_purpose3.reset_index(inplace=True)  # 날짜 인덱스 > 컬럼으로 변경

In [ ]:
cpi_total['year'] = cpi_total['index'].str[:4].astype('int')
cpi_total['month'] = cpi_total['index'].str[5:].astype('int')
cpi_total.drop(columns=['index'], inplace=True, axis=1)

cpi_purpose3['year'] = cpi_purpose3['index'].str[:4].astype('int')
cpi_purpose3['month'] = cpi_purpose3['index'].str[5:].astype('int')
cpi_purpose3.drop(columns=['index'], inplace=True, axis=1)

## Add to Training Data

In [ ]:
df_train = pd.merge(df_train, df_cpi1, how='left', on=['year', 'month'])
df_train = pd.merge(df_train, df_cpi2, how='left', on=['year', 'month'])

In [ ]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

---

# Final

## One-hot Encoding

In [ ]:
one_hot = ['CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1', 'CATEGORY_2', 'P_NAME']

In [ ]:
df_train = pd.get_dummies(df_train, columns=one_hot)

## Drop Columns

In [ ]:
drop = ['REG_DATE', 'P_TYPE', 'P_IMPORT_TYPE']

In [ ]:
df_train.drop(columns = drop, inplace=True)

In [ ]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

## Save Files

In [ ]:
def save_file(df, file_name):
    df.to_csv(os.path.join(root, file_name), encoding='utf-8', index=False)

In [ ]:
save_file(df_weather, 'preprocessed_weather.csv')  # 제조국 날씨
save_file(df_oil, 'preprocessed_oil.csv')  # 원유 종가
save_file(df_weather_kr, 'preprocessed_weather_korea.csv')  # 한국 날씨
save_file(df_cpi1, 'preprocessed_cpi_region.csv')  # 전체 소비자물가지수
save_file(df_cpi2, 'preprocessed_cpi_purpose.csv')  # 음식/음식서비스 소비자물가지수
save_file(df_train, 'preprocessed_train.csv')  # 최종 df